# 自动微分

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
x.requires_grad_(True)  # 等价于 `x = torch.arange(4.0, requires_grad=True)`
x.grad  # 默认值是None

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
y.backward()  # 通过调用反向传播函数来自动计算y关于x的梯度
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

In [6]:
x.grad.zero_()  # 梯度清零
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 非标量变量的反向传播

In [7]:
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## 分离计算

In [8]:
x.grad.zero_()  # 需要清零之前的梯度
y = x * x
u = y.detach()  # 分离y，使得我们可以对y进行操作而不会影响到梯度
z = u * x  # z关于x的梯度在y被分离后,y就会被当做常数处理

In [9]:
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [10]:
x.grad.zero_()
y.sum().backward() # 调用反向传播函数
x.grad == 2 * x

tensor([True, True, True, True])

## Python控制流的梯度计算
这个函数的好处是，它允许我们在定义计算图的时候使用Python的控制流。

In [11]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [12]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [13]:
a.grad == d / a

tensor(True)